In [1]:
from openai import OpenAI
import dotenv
import rich
dotenv.load_dotenv('.env')
import mrf
import mrf_data
import tiktoken
import json
import llm
encoding = tiktoken.encoding_for_model('gpt-4o')
from importlib import reload
import torch
import numpy as np


In [31]:
example = json.load(open('example_question.json'))

# separate 'Text' and the rest of the entries

text = example['Text']
example.pop('Text')
js = json.dumps(example)

In [36]:
prompt = []

system = "Translate the provided JSON data into a question sentence in English. Follow the provided example. Include keyword Airbnb."

user_1 = js
assistant_1 = text

prompt.append({'role': 'system', 'content': system})
prompt.append({'role': 'user', 'content': user_1})
prompt.append({'role': 'assistant', 'content': assistant_1})

print(prompt)



class question_translate():
    def __init__(self, prompt):
        self.client = OpenAI()
        self.prompt = prompt
        self.model = 'gpt-3.5-turbo'
        
    
    def get_text(self, js_question):
        tmp = self.prompt.copy()
        if type(js_question) == dict:
            js_question = json.dumps(js_question)
        tmp.append({'role': 'user', 'content': js_question})
        response = self.client.chat.completions.create(
            model=self.model,
            response_format={ "type": "text" },
            temperature=0,
            messages=tmp
        )
        return response.choices[0].message.content
    
    def put_new(self, js_path, out_path=None):

        js = json.load(open(js_path)).copy()
        text = self.get_text(js)
        js['Text'] = text

        if out_path:
            json.dump(js, open(out_path, 'w'), indent=2)
        else:
            out_path = js_path[:-5] + 'q.json'
            json.dump(js, open('output.json', 'w'), indent=2)

        return text



[{'role': 'system', 'content': 'Translate the provided JSON data into a question sentence in English. Follow the provided example. Include keyword Airbnb.'}, {'role': 'user', 'content': '{"Target": [{"Name": "price", "Type": "float"}], "Condition": [{"Name": "room_type", "Type": "str", "Value": ["Private room"]}, {"Name": "beds", "Type": "int", "Value": ["1", "2", "3"]}]}'}, {'role': 'assistant', 'content': 'What is the nightly price of a aribnb listing that is a private room in a property with at most 3 beds?'}]


In [37]:
trans = question_translate(prompt)
test_example = json.load(open('test_question.json'))


In [38]:
prompt

[{'role': 'system',
  'content': 'Translate the provided JSON data into a question sentence in English. Follow the provided example. Include keyword Airbnb.'},
 {'role': 'user',
  'content': '{"Target": [{"Name": "price", "Type": "float"}], "Condition": [{"Name": "room_type", "Type": "str", "Value": ["Private room"]}, {"Name": "beds", "Type": "int", "Value": ["1", "2", "3"]}]}'},
 {'role': 'assistant',
  'content': 'What is the nightly price of a aribnb listing that is a private room in a property with at most 3 beds?'}]

In [39]:
trans.put_new('test_question.json')

'What is the nightly price of an Airbnb villa listing in New York or Los Angeles with 2 or more bathrooms?'

In [24]:
trans.get_text(test_example)

'What is the nightly price of an Airbnb listing for a villa with 2 or more bathrooms in either New York or Los Angeles?'